In [182]:
import importlib
import pandas as pd
from bs4 import BeautifulSoup
from src import scrapers
importlib.reload(scrapers)
test_scraper = scrapers.PfrScraper(
    '/Users/Jared/OneDrive - Sportsbook Science/data/raw/nfl')

In [160]:
test_scraper.request_season_summaries(range(1970, 1980))

1970 Season Summary already saved locally, skipping
1971 Season Summary already saved locally, skipping
1972 Season Summary already saved locally, skipping
1973 Season Summary already saved locally, skipping
1974 Season Summary already saved locally, skipping
1975 Season Summary already saved locally, skipping
1976 Season Summary already saved locally, skipping
1977 Season Summary already saved locally, skipping
1978 Season Summary already saved locally, skipping
1979 Season Summary already saved locally, skipping


In [173]:
sum_html = test_scraper.load_season_summary(1993)
sum_df = test_scraper.scrape_season_summary(sum_html)
print(sum_df['week_summary_links'][0].split(','))
sum_df.head()


['/years/1993/week_1.htm', '/years/1993/week_2.htm', '/years/1993/week_3.htm', '/years/1993/week_4.htm', '/years/1993/week_5.htm', '/years/1993/week_6.htm', '/years/1993/week_7.htm', '/years/1993/week_8.htm', '/years/1993/week_9.htm', '/years/1993/week_10.htm', '/years/1993/week_11.htm', '/years/1993/week_12.htm', '/years/1993/week_13.htm', '/years/1993/week_14.htm', '/years/1993/week_15.htm', '/years/1993/week_16.htm', '/years/1993/week_17.htm', '/years/1993/week_18.htm', '/years/1993/week_19.htm', '/years/1993/week_20.htm', '/years/1993/week_21.htm', '/years/1993/week_22.htm']


,season,num_weeks,week_summary_links
0,1993,22,"/years/1993/week_1.htm,/years/1993/week_2.htm,..."


In [180]:
week_sum_html = test_scraper.load_week_summary(season=2021, week=2)
week_sum_df = test_scraper.scrape_week_summary(week_sum_html)
week_sum_df.head()


,num_games,game_summary_links
0,16,"/boxscores/202109160was.htm,/boxscores/2021091..."


In [184]:
game_links = week_sum_df['game_summary_links'][0].split(',')
game_html = test_scraper.get_with_requests(test_scraper.base_url+game_links[0])

In [186]:
game_soup = BeautifulSoup(game_html, 'html.parser')
# Game Summary Info
# Get Linescore
linescore = str(
    game_soup.find('table', class_='linescore nohover stats_table no_freeze'))
linescore_df = pd.read_html(linescore)[0]
entry = {
    'away_team': linescore_df['Unnamed: 1'][0],
    'away_team_score_q1': linescore_df['1'][0],
    'away_team_score_q2': linescore_df['2'][0],
    'away_team_score_q3': linescore_df['3'][0],
    'away_team_score_q4': linescore_df['4'][0],
    'away_team_score_final': linescore_df['Final'][0],
    'home_team': linescore_df['Unnamed: 1'][1],
    'home_team_score_q1': linescore_df['1'][1],
    'home_team_score_q2': linescore_df['2'][1],
    'home_team_score_q3': linescore_df['3'][1],
    'home_team_score_q4': linescore_df['4'][1],
    'home_team_score_final': linescore_df['Final'][1]
}
# Check Overtime
if 'OT' in linescore_df.columns:
    entry['overtime_played'] = True
    entry['away_team_score_overtime'] = linescore_df['OT'][0]
    entry['home_team_score_overtime'] = linescore_df['OT'][1]
else:
    entry['overtime_played'] = False
    entry['away_team_score_overtime'] = 0
    entry['home_team_score_overtime'] = 0

print(linescore_df.head())




{'away_team': 'New York Giants', 'away_team_score_q1': 7, 'away_team_score_q2': 3, 'away_team_score_q3': 10, 'away_team_score_q4': 9, 'away_team_score_final': 29, 'home_team': 'Washington Football Team', 'home_team_score_q1': 0, 'home_team_score_q2': 14, 'home_team_score_q3': 3, 'home_team_score_q4': 13, 'home_team_score_final': 30, 'overtime_played': False, 'away_team_score_overtime': 0, 'home_team_score_overtime': 0}


,Unnamed: 0,Unnamed: 1,1,2,3,4,Final
0,via Sports Logos.net About logos,New York Giants,7,3,10,9,29
1,via Sports Logos.net About logos,Washington Football Team,0,14,3,13,30
